In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile5 = pd.read_pickle("BBB_Train_Decile5.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile5.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile5, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 10s 3ms/step - loss: 0.4058 - acc: 0.8416 - val_loss: 0.3863 - val_acc: 0.8394
Epoch 36/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4023 - acc: 0.8422 - val_loss: 0.3860 - val_acc: 0.8433
Epoch 37/75
3087/3087 [==============================] - 11s 3ms/step - loss: 0.4042 - acc: 0.8507 - val_loss: 0.3866 - val_acc: 0.8459
Epoch 38/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4031 - acc: 0.8432 - val_loss: 0.3848 - val_acc: 0.8420
Epoch 39/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4063 - acc: 0.8435 - val_loss: 0.3883 - val_acc: 0.8433
Epoch 40/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4046 - acc: 0.8448 - val_loss: 0.3835 - val_acc: 0.8433
Epoch 41/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4076 - acc: 0.8406 - val_loss: 0.3843 - val_acc: 0.8459
Epoch 42/75
3087/3087 [==============================] - 11s


Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 19/75
3087/3087 [==============================] - 11s 3ms/step - loss: 0.3662 - acc: 0.8510 - val_loss: 0.3736 - val_acc: 0.8459
Epoch 20/75
3087/3087 [==============================] - 11s 3ms/step - loss: 0.3749 - acc: 0.8439 - val_loss: 0.3737 - val_acc: 0.8446
Epoch 21/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3706 - acc: 0.8432 - val_loss: 0.3736 - val_acc: 0.8459
Epoch 22/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3705 - acc: 0.8500 - val_loss: 0.3737 - val_acc: 0.8459
Epoch 23/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3772 - acc: 0.8468 - val_loss: 0.3736 - val_acc: 0.8459

Epoch 00023: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 24/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3748 - acc: 0.8452 - val_loss: 0.3738 - val_acc: 0.8459
Epoch 25/75
3087/3087 [==============

3087/3087 [==============================] - 10s 3ms/step - loss: 0.5517 - acc: 0.7204 - val_loss: 0.4832 - val_acc: 0.7811
Epoch 3/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4822 - acc: 0.8040 - val_loss: 0.4408 - val_acc: 0.8316
Epoch 4/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4521 - acc: 0.8267 - val_loss: 0.4202 - val_acc: 0.8381
Epoch 5/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4284 - acc: 0.8364 - val_loss: 0.4075 - val_acc: 0.8368
Epoch 6/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4234 - acc: 0.8264 - val_loss: 0.4011 - val_acc: 0.8355
Epoch 7/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4145 - acc: 0.8319 - val_loss: 0.3969 - val_acc: 0.8355
Epoch 8/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4163 - acc: 0.8380 - val_loss: 0.3927 - val_acc: 0.8355
Epoch 9/100
3087/3087 [==============================] - 11s

Epoch 61/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3721 - acc: 0.8490 - val_loss: 0.3771 - val_acc: 0.8446
Epoch 62/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3728 - acc: 0.8429 - val_loss: 0.3772 - val_acc: 0.8446
Epoch 63/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3737 - acc: 0.8439 - val_loss: 0.3772 - val_acc: 0.8433
Epoch 64/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3637 - acc: 0.8503 - val_loss: 0.3773 - val_acc: 0.8446
Epoch 65/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3662 - acc: 0.8445 - val_loss: 0.3773 - val_acc: 0.8446
Epoch 66/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3677 - acc: 0.8474 - val_loss: 0.3772 - val_acc: 0.8446
Epoch 67/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.3705 - acc: 0.8503 - val_loss: 0.3772 - val_acc: 0.8446
Epoch 68/100
3087/3087 [=================

...

3087/3087 [==============================] - 8s 3ms/step - loss: 0.4345 - acc: 0.8358 - val_loss: 0.4143 - val_acc: 0.8368
Epoch 24/125
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4314 - acc: 0.8328 - val_loss: 0.4126 - val_acc: 0.8381
Epoch 25/125
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4279 - acc: 0.8397 - val_loss: 0.4097 - val_acc: 0.8407
Epoch 26/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4225 - acc: 0.8380 - val_loss: 0.4084 - val_acc: 0.8381
Epoch 27/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.4236 - acc: 0.8422 - val_loss: 0.4075 - val_acc: 0.8342
Epoch 28/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.4217 - acc: 0.8374 - val_loss: 0.4062 - val_acc: 0.8368
Epoch 29/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.4199 - acc: 0.8367 - val_loss: 0.4040 - val_acc: 0.8355
Epoch 30/125
3087/3087 [==============================]

3087/3087 [==============================] - 11s 3ms/step - loss: 0.3969 - acc: 0.8481 - val_loss: 0.3851 - val_acc: 0.8433
Epoch 83/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.4013 - acc: 0.8426 - val_loss: 0.3845 - val_acc: 0.8459
Epoch 84/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3990 - acc: 0.8458 - val_loss: 0.3843 - val_acc: 0.8459
Epoch 85/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3998 - acc: 0.8452 - val_loss: 0.3842 - val_acc: 0.8459
Epoch 86/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3941 - acc: 0.8507 - val_loss: 0.3844 - val_acc: 0.8446
Epoch 87/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.4027 - acc: 0.8452 - val_loss: 0.3840 - val_acc: 0.8484
Epoch 88/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3956 - acc: 0.8400 - val_loss: 0.3840 - val_acc: 0.8484
Epoch 89/125
3087/3087 [==============================

3087/3087 [==============================] - 7s 2ms/step - loss: 0.3805 - acc: 0.8413 - val_loss: 0.3778 - val_acc: 0.8510
Epoch 17/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3834 - acc: 0.8371 - val_loss: 0.3769 - val_acc: 0.8497
Epoch 18/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3731 - acc: 0.8429 - val_loss: 0.3935 - val_acc: 0.8472
Epoch 19/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3755 - acc: 0.8458 - val_loss: 0.3805 - val_acc: 0.8484

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 20/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3789 - acc: 0.8432 - val_loss: 0.3787 - val_acc: 0.8484
Epoch 21/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3723 - acc: 0.8435 - val_loss: 0.3788 - val_acc: 0.8497
Epoch 22/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3689 - acc: 0.8400 - val_loss: 0.37

3087/3087 [==============================] - 4s 1ms/step - loss: 0.3745 - acc: 0.8432 - val_loss: 0.3790 - val_acc: 0.8497
Epoch 75/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3772 - acc: 0.8439 - val_loss: 0.3788 - val_acc: 0.8497
Epoch 76/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3658 - acc: 0.8432 - val_loss: 0.3788 - val_acc: 0.8497
Epoch 77/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3701 - acc: 0.8448 - val_loss: 0.3789 - val_acc: 0.8497
Epoch 78/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3752 - acc: 0.8435 - val_loss: 0.3788 - val_acc: 0.8497
Epoch 79/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3731 - acc: 0.8439 - val_loss: 0.3789 - val_acc: 0.8497
Epoch 80/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3701 - acc: 0.8422 - val_loss: 0.3789 - val_acc: 0.8497
Epoch 81/100
3087/3087 [==============================] - 4s 

Epoch 33/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3780 - acc: 0.8455 - val_loss: 0.3779 - val_acc: 0.8446
Epoch 34/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3767 - acc: 0.8455 - val_loss: 0.3777 - val_acc: 0.8446
Epoch 35/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3832 - acc: 0.8461 - val_loss: 0.3775 - val_acc: 0.8446
Epoch 36/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3781 - acc: 0.8465 - val_loss: 0.3776 - val_acc: 0.8446
Epoch 37/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3850 - acc: 0.8442 - val_loss: 0.3779 - val_acc: 0.8446
Epoch 38/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3782 - acc: 0.8439 - val_loss: 0.3780 - val_acc: 0.8446
Epoch 39/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.3894 - acc: 0.8409 - val_loss: 0.3779 - val_acc: 0.8446
Epoch 40/50
3087/3087 [==============================] 

In [ ]:
0.8297